In [ ]:
# TPU 사용시
VERSION = "nightly" #@param ["20200220","nightly", "xrt==1.15.0"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

In [ ]:
# colab gpu(tpu) 사용
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 기본적으로 필요한 툴들 가져오기
import pandas as pd
import torch
# TPU 사용시
# import torch_xla
# import torch_xla.core.xla_model as xm

In [ ]:
# 모델, 학습 관련 변수들 define

# TPU 사용시
# device = xm.xla_device()
# GPU 사용시
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = 3200
hidden_size = 256
num_attention_heads = 4
num_encoder_layers = 6
num_decoder_layers = 6
intermediate_size = 512
dropout = 0.1
PATH = "webtoon_comment_model.pt"

# 3차 학습
split = 0.1
random_seed = 42

In [ ]:
# 학습된 모델을 저장하는 함수
def save(model,path):
  torch.save(model.state_dict(), PATH)

In [ ]:
# 모델 불러오기
def load_model(path):
  model = Model(vocab_size,hidden_size,num_attention_heads,num_encoder_layers,num_decoder_layers,intermediate_size,dropout)
  model.load_state_dict(torch.load(path))
  return model.to(device)

In [ ]:
# 토크나이저 define
from collections import Counter, defaultdict

SPECIAL_TOKENS = ['<unk>', '<pad>', '<sos>', '<eos>']


# 음절 단위로 vocab을 형성한다. 
class CharTokenizer(object):
    def __init__(self, i2c):
        self.init(i2c)

    def __len__(self):
        return len(self.vocab)

    def __call__(self, sent):
        return [self.vocab[c] for c in sent]

    def init(self, i2c):
        self.i2c = i2c
        self.vocab = defaultdict(int)
        self.vocab.update({c: i for i, c in enumerate(i2c)})

    @classmethod
    def from_strings(cls, strings, vocab_size): # 전체 dataset으로 vocab을 만들어 준다.
        char_counter = Counter()
        for x in strings:
            char_counter.update(x)
        i2c = SPECIAL_TOKENS # vocab의 맨 앞 4개는 special 토큰으로 채워줌
        i2c += [c for c, _ in char_counter.most_common(vocab_size - len(SPECIAL_TOKENS))] # 나머지는 dataset에 있는 단어로 채움 
        return cls(i2c) # init을 호출하면서 return 

In [ ]:
# Data Loader Define

In [ ]:
# 모델 class define
import math

import torch
import torch.nn as nn
from torch.nn import Transformer

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:x.size(0), :]


class TransformerModel(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_attention_heads, num_encoder_layers, num_decoder_layers, intermediate_size, dropout=0.1):
        super(TransformerModel, self).__init__()

        self.token_embeddings = nn.Embedding(vocab_size, hidden_size)
        self.position_embeddings = PositionalEncoding(hidden_size)
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(p=dropout)

        self.transformer = Transformer(
            d_model=hidden_size,
            nhead=num_attention_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=intermediate_size,
            dropout=dropout,
        )

        self.decoder_embeddings = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.decoder_embeddings.weight = self.token_embeddings.weight

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.token_embeddings.weight.data.uniform_(-initrange, initrange)
        self.decoder_embeddings.bias.data.zero_()
        self.decoder_embeddings.weight.data.uniform_(-initrange, initrange)

    def forward(self, src=None, tgt=None, memory=None, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        src_embeddings = self.token_embeddings(src) * math.sqrt(self.hidden_size) + self.position_embeddings(src)
        src_embeddings = self.dropout(src_embeddings)

        if src_key_padding_mask is not None:
            src_key_padding_mask = src_key_padding_mask.t()

        if tgt is None:  # encode
            memory = self.transformer.encoder(src_embeddings, src_key_padding_mask=src_key_padding_mask)
            return memory

class Model(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_attention_heads, num_encoder_layers, num_decoder_layers, intermediate_size, dropout=0.1):
        super(Model, self).__init__()
        self.transformer = TransformerModel(vocab_size, hidden_size, num_attention_heads, num_encoder_layers, num_decoder_layers, intermediate_size, dropout)
        self.decoder_embeddings = nn.Linear(hidden_size,3)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, src=None, tgt=None, memory=None, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        output = self.transformer(src, tgt, memory, src_key_padding_mask.t(), tgt_key_padding_mask, memory_key_padding_mask)
        output = output[-1, : , :]
        output = self.decoder_embeddings(output)
        output = self.softmax(output)
        return output

In [ ]:
# Define TextDataset & collate functionimport torch
from torch.utils.data import Dataset,DataLoader, RandomSampler, SequentialSampler

class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


def collate_fn(data, tokenizer, max_seq_length=None):
    src_token_ids = [tokenizer(x[0]) for x in data]
    label = [x[1] for x in data]

    src_max_seq_length = max([len(x) for x in src_token_ids])
    if max_seq_length and max_seq_length < src_max_seq_length:
        src_max_seq_length = max_seq_length

    src_padded = []
    src_padding_mask = []
    for src in src_token_ids:
        src = src[:src_max_seq_length]
        src_pad_length = src_max_seq_length - len(src)
        src_padded.append(src + [1] * src_pad_length)
        src_padding_mask.append([1] * len(src) + [0] * src_pad_length)

    src_padded = torch.tensor(src_padded).t().contiguous()
    src_padding_mask = torch.tensor(src_padding_mask).bool()

    return src_padded, src_padding_mask,label

In [ ]:
# Define Trainer
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

def train(model, optimizer, scheduler, criterion, tokenizer, train_data, valid_data=None, new_path=None):
  loss_value = float("inf")
  acc_max = 0.0
  model.train()
  train_dataset = TextDataset(train_data)
  train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=512, num_workers=2,collate_fn=lambda x: collate_fn(x, tokenizer, 128))
  
  if valid_data is not None:
    valid_dataset = TextDataset(valid_data)
    valid_dataloader = DataLoader(valid_dataset, sampler=SequentialSampler(valid_dataset), batch_size=256, num_workers=2,collate_fn=lambda x: collate_fn(x, tokenizer, 128))
    
  for epoch in range(10000):
    print("========================================")
    # Get Prediction Rate
    if valid_data is not None:
      model.eval()
      correct = 0
      total = 0
      with torch.no_grad():
        for i,batch in enumerate(valid_dataloader):
          src,src_mask,label = batch
          src = src.to(device)
          src_mask = src_mask.to(device)
          label = torch.IntTensor(label).to(device)

          output = model(src=src,src_key_padding_mask=~src_mask)
          _, predicted = torch.max(output.data, 1)
          total += label.size(0)
          correct += (predicted == label).sum().item()
          accuracy = (correct/total)*100
        print("Prediction probability  : ",accuracy,"%")  
        print("==================================================")
        
        if acc_max < accuracy:
          acc_max = accuracy
          save(model,PATH)
    # Train model
    model.train()
    for i,batch in enumerate(train_dataloader):
      src,src_mask,label = batch
      src = src.to(device)
      src_mask = src_mask.to(device)
      label = torch.LongTensor(label).to(device)

      output = model(src=src,src_key_padding_mask=~src_mask)
      loss = criterion(output,label)
      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), 0.5)
      optimizer.step()
      scheduler.step()
      print(epoch," 번째 epoch,", i," 번째 배치 의 loss값 : ",loss.item())
      if valid_data is None and loss_value > loss.item():
        loss_value = loss.item()
        if new_path is None:
          save(model,PATH)
        else:
          save(model,new_path)

In [ ]:
# Get Prediction Rate
def evaluate(model, tokenizer, valid_data):
  model.eval()
  valid_dataset = TextDataset(valid_data)
  valid_dataloader = DataLoader(valid_dataset, sampler=SequentialSampler(valid_dataset), batch_size=256, num_workers=2,collate_fn=lambda x: collate_fn(x, tokenizer, 128))
  
  correct = 0
  total = 0
  with torch.no_grad():
    for i,batch in enumerate(valid_dataloader):
      src,src_mask,label = batch
      src = src.to(device)
      src_mask = src_mask.to(device)
      label = torch.IntTensor(label).to(device)

      output = model(src=src,src_key_padding_mask=~src_mask)
      
      _, predicted = torch.max(output.data, 1)
      total += label.size(0)
      correct += (predicted == label).sum().item()
    print("Prediction probability  : ",(correct/total)*100,"%")  

In [ ]:
# tokenizer 생성

# 결측치 제거
df = pd.read_excel('webtoon_comments_1000_labeled.xlsx')
df = df.dropna()
# 총 char 수 새기
comment_sents = df['comment'].values.tolist()
char_set = set()
for cmt in comment_sents:
  for char in cmt:
    char_set.add(char)
print(len(char_set))

# tokneizer 생성 : 총 char수에서 217개를 뺀 수를 vocab size로 지정한다.(자주 등장하는 cahr만 사용하기 위함)
tokenizer = CharTokenizer([])
tokenizer= CharTokenizer.from_strings(comment_sents, len(char_set)-217)

3417


In [ ]:
########
#1차 학습
#########

In [ ]:
# 데이터 가져옴(앞에서 1000개만)
df = pd.read_excel('webtoon_comments_1000_labeled.xlsx').dropna()
df = df.loc[:1000]
comment = df['comment'].tolist()
label = df['label'].tolist()
data = [(comment[i],label[i]) for i,_ in enumerate(comment)]

In [ ]:
# 모델 생성
model = Model(vocab_size,hidden_size,num_attention_heads,num_encoder_layers,num_decoder_layers,intermediate_size,dropout)
model = model.to(device)

In [ ]:
#model = load_model(PATH).to(device)

In [ ]:
# 모델 optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=5e-6,betas=eval("(0.9, 0.98)"),eps=1e-9,weight_decay=1e-9)
lr_lambda = lambda x: x / 200 if x <= 200 else (x / 200) ** -0.5
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
criterion = nn.CrossEntropyLoss()

In [ ]:
# 모델 학습

train(model,optimizer,scheduler,criterion,tokenizer,data,None)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
8333  번째 epoch, 1  번째 배치 의 loss값 :  0.6142398118972778
8334  번째 epoch, 0  번째 배치 의 loss값 :  0.6257013082504272
8334  번째 epoch, 1  번째 배치 의 loss값 :  0.6273400783538818
8335  번째 epoch, 0  번째 배치 의 loss값 :  0.6095179319381714
8335  번째 epoch, 1  번째 배치 의 loss값 :  0.6387167572975159
8336  번째 epoch, 0  번째 배치 의 loss값 :  0.6347635984420776
8336  번째 epoch, 1  번째 배치 의 loss값 :  0.6084724068641663
8337  번째 epoch, 0  번째 배치 의 loss값 :  0.6226564645767212
8337  번째 epoch, 1  번째 배치 의 loss값 :  0.6287979483604431
8338  번째 epoch, 0  번째 배치 의 loss값 :  0.6167486310005188
8338  번째 epoch, 1  번째 배치 의 loss값 :  0.6275178790092468
8339  번째 epoch, 0  번째 배치 의 loss값 :  0.6350641846656799
8339  번째 epoch, 1  번째 배치 의 loss값 :  0.6131209135055542
8340  번째 epoch, 0  번째 배치 의 loss값 :  0.6244726181030273
8340  번째 epoch, 1  번째 배치 의 loss값 :  0.6227273344993591
8341  번째 epoch, 0  번째 배치 의 loss값 :  0.6258946061134338
8341  번째 epoch, 1  번째 배치 의 loss값 :  0.6194049715995789
8342  번째 epoch, 0  번째 배치 의 lo

In [ ]:
# 모델 가져오고 평가하기
model_eval = load_model(PATH)
evaluate(model_eval,tokenizer, data)

Prediction probability  :  93.30000000000001 %


In [ ]:
#########
#2차 데이터라벨링
#########

In [ ]:
# 1000번 이후의 데이터들을 예측해 새로이 라벨링 하고 저장하기
def label_all(model,tokenizer, valid_data): 
  model.eval()
  valid_dataset = TextDataset(valid_data)
  valid_dataloader = DataLoader(valid_dataset, sampler=SequentialSampler(valid_dataset), batch_size=64, num_workers=2,collate_fn=lambda x: collate_fn(x, tokenizer, 128))
  labeled = []
  with torch.no_grad():
    for i,batch in enumerate(valid_dataloader):
      src,src_mask,label = batch
      src = src.to(device)
      src_mask = src_mask.to(device)

      output = model(src=src,src_key_padding_mask=~src_mask)
      _, predicted = torch.max(output.data, 1)
      labeled += predicted.tolist()
  return labeled

In [ ]:
# 데이터 가져옴(1000 개 이후)
df = pd.read_excel('webtoon_comments_1000_labeled.xlsx').dropna()
df = df.loc[1001:]
comment = df['comment'].tolist()
label = df['label'].tolist()
data = [(comment[i],label[i]) for i,_ in enumerate(comment)]

In [ ]:
# 모델 생성
model_eval = load_model(PATH)
model_eval = model_eval.to(device)

In [ ]:
# 예측해서 새로 라벨링 하기
label = label_all(model_eval,tokenizer, data)

df['label'] = label
df_1000 = pd.read_excel('webtoon_comments_1000_labeled.xlsx').dropna()
df_1000 = df_1000.loc[:1000]
comment_1000 = df_1000['comment'].tolist()
label_1000 = df_1000['label'].tolist()

comment = comment_1000 + comment
label = label_1000 + label

df = pd.DataFrame(list(zip(comment,label)),columns=['comment','label'] )
df.to_csv('webtoon_comments_labeled.csv',sep=',',encoding='utf-16')

In [ ]:
#########
#3차 학습
#########

In [ ]:
# 데이터 가져옴(전체 데이터)
df = pd.read_csv('webtoon_comments_labeled.csv',encoding='utf-16')
comment = df['comment'].tolist()
label = df['label'].tolist()
data = [(comment[i],label[i]) for i,_ in enumerate(comment)]

In [ ]:
# 학습, 평가 데이터셋을 나눔
from sklearn.model_selection import train_test_split

valid_data,train_data = train_test_split(data, test_size=1-split, random_state=random_seed)

In [ ]:
# test set 라벨 분포
zero = 0
one = 0
two = 0
for _,label in valid_data:
  if label==0:
    zero+=1
  elif label==1:
    one+=1;
  elif label==2:
    two+=1;
print("zero : {}, one : {}, two : {}".format(zero,one,two))


zero : 3367, one : 360, two : 690


In [ ]:
# 모델 생성

model = load_model(PATH)
# model = load_model("webtoon_comment_model_93.pt") 파일 충돌때문에 이름 잠깐 바꿈
model = model.to(device)

In [ ]:
# 모델 optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=5e-6,betas=eval("(0.9, 0.98)"),eps=1e-9,weight_decay=1e-9)
lr_lambda = lambda x: x / 50 if x <= 50 else (x / 50) ** -0.5
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
criterion = nn.CrossEntropyLoss()

In [ ]:
# 모델 학습 및 best model 저장
train(model,optimizer,scheduler,criterion,tokenizer,train_data,valid_data)

Prediction probability  :  99.88680099615124 %
0  번째 epoch, 0  번째 배치 의 loss값 :  0.5619270205497742
0  번째 epoch, 1  번째 배치 의 loss값 :  0.5624449253082275
0  번째 epoch, 2  번째 배치 의 loss값 :  0.5634051561355591
0  번째 epoch, 3  번째 배치 의 loss값 :  0.5683316588401794
0  번째 epoch, 4  번째 배치 의 loss값 :  0.5600712895393372
0  번째 epoch, 5  번째 배치 의 loss값 :  0.5633127093315125
0  번째 epoch, 6  번째 배치 의 loss값 :  0.5667333006858826
0  번째 epoch, 7  번째 배치 의 loss값 :  0.5624924302101135
0  번째 epoch, 8  번째 배치 의 loss값 :  0.5587334632873535
0  번째 epoch, 9  번째 배치 의 loss값 :  0.5635855793952942
0  번째 epoch, 10  번째 배치 의 loss값 :  0.5640518069267273
0  번째 epoch, 11  번째 배치 의 loss값 :  0.5626150965690613
0  번째 epoch, 12  번째 배치 의 loss값 :  0.5605223178863525
0  번째 epoch, 13  번째 배치 의 loss값 :  0.5663952231407166
0  번째 epoch, 14  번째 배치 의 loss값 :  0.5634337663650513
0  번째 epoch, 15  번째 배치 의 loss값 :  0.5589718222618103
0  번째 epoch, 16  번째 배치 의 loss값 :  0.5602514147758484
0  번째 epoch, 17  번째 배치 의 loss값 :  0.5604450702667236
0  번째 epo

KeyboardInterrupt: ignored

In [ ]:
# 라벨링 된 데이터로 학습하는 모델 생성

model = Model(vocab_size,hidden_size,num_attention_heads,num_encoder_layers,num_decoder_layers,intermediate_size,dropout)
model = model.to(device)

In [ ]:
# 모델 optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=5e-6,betas=eval("(0.9, 0.98)"),eps=1e-9,weight_decay=1e-9)
lr_lambda = lambda x: x / 50 if x <= 50 else (x / 50) ** -0.5
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
criterion = nn.CrossEntropyLoss()

In [ ]:
# 모델 학습 및 best model 저장
new_path = "webtoon_comment_new_model.pt"
train(model,optimizer,scheduler,criterion,tokenizer,train_data,valid_data,new_path)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Prediction probability  :  86.91419515508264 %
957  번째 epoch, 0  번째 배치 의 loss값 :  0.681989848613739
957  번째 epoch, 1  번째 배치 의 loss값 :  0.7114086747169495
957  번째 epoch, 2  번째 배치 의 loss값 :  0.6849012970924377
957  번째 epoch, 3  번째 배치 의 loss값 :  0.693969190120697
957  번째 epoch, 4  번째 배치 의 loss값 :  0.6765713691711426
957  번째 epoch, 5  번째 배치 의 loss값 :  0.6810615658760071
957  번째 epoch, 6  번째 배치 의 loss값 :  0.6889137029647827
957  번째 epoch, 7  번째 배치 의 loss값 :  0.6583131551742554
957  번째 epoch, 8  번째 배치 의 loss값 :  0.6826228499412537
957  번째 epoch, 9  번째 배치 의 loss값 :  0.668475866317749
957  번째 epoch, 10  번째 배치 의 loss값 :  0.6974194049835205
957  번째 epoch, 11  번째 배치 의 loss값 :  0.6601715087890625
957  번째 epoch, 12  번째 배치 의 loss값 :  0.7063159942626953
957  번째 epoch, 13  번째 배치 의 loss값 :  0.6682779788970947
957  번째 epoch, 14  번째 배치 의 loss값 :  0.7089589834213257
957  번째 epoch, 15  번째 배치 의 loss값 :  0.6567989587783813
957  번째 epoch, 16  번째 배치 의 loss값 :  0.686724066734